CONFIGURATION

In [ ]:
#Installation des dépendances en amont si non-présentes dans l'environnement
!pip install numpy pandas scikit-learn

In [ ]:
#Importation des librairies communes (peu prendre du temps)
import os
import sys
import numpy
import pandas as pd

In [ ]:
#Chemin menant aux données
data_path = os.path.join(os.getcwd(), "source")
#création du dictionnaire
data_dict = dict()

In [ ]:
# Boucle pour récupérer tous les fichiers CSV dans le dossier et les ajouter au dictionnaire
for file in os.listdir(data_path):
    if file.endswith(".csv"):
        file_path = os.path.join(data_path, file)
        dict_key = file.split('.')[0]  # Nom du fichier sans l'extension .csv
        data_dict[dict_key] = pd.read_csv(file_path)

data_dict.keys()

FONCTIONS UTILES

In [ ]:
def games_of_the_season(df_games, club_id, saison, lieu='tous'):
    """
    Récupère tous les matchs d'une saison pour un club spécifié, ou les 10 derniers matchs si moins de 5 matchs dans la saison.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.
    saison (str) : Saison à filtrer.
    lieu (str) : 'domicile', 'exterieur', ou 'tous' pour filtrer les matchs.
    
    Retourne:
    DataFrame : Les matchs demandés avec les informations détaillées.
    """
    # Filtrage initial par saison et lieu
    if lieu == 'domicile':
        matchs_du_club = df_games[(df_games['home_club_id'] == club_id) & (df_games['season'] == saison)]
    elif lieu == 'exterieur':
        matchs_du_club = df_games[(df_games['away_club_id'] == club_id) & (df_games['season'] == saison)]
    else:
        matchs_du_club = df_games[((df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)) & (df_games['season'] == saison)]
    
    # Vérification du nombre de matchs
    if len(matchs_du_club) < 5:
        # Prendre les 10 derniers matchs toutes saisons confondues si moins de 5 matchs dans la saison
        matchs_du_club = df_games[(df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)]
        matchs_du_club = matchs_du_club.sort_values(by='date', ascending=False).head(10)
    else:
        # Trier par date
        matchs_du_club = matchs_du_club.sort_values(by='date', ascending=False)

    # Ajout des informations supplémentaires
    matchs_du_club['adversaire_id'] = matchs_du_club.apply(lambda row: row['away_club_id'] if row['home_club_id'] == club_id else row['home_club_id'], axis=1)
    matchs_du_club['lieu'] = matchs_du_club.apply(lambda row: 'domicile' if row['home_club_id'] == club_id else 'exterieur', axis=1)
    matchs_du_club['résultat'] = matchs_du_club.apply(lambda row: 'victoire' if (row['home_club_id'] == club_id and row['home_club_goals'] > row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] > row['home_club_goals']) else 'défaite' if (row['home_club_id'] == club_id and row['home_club_goals'] < row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] < row['home_club_goals']) else 'nul', axis=1)

    return matchs_du_club[['date', 'season', 'home_club_id', 'away_club_id', 'adversaire_id', 'lieu', 'home_club_goals', 'away_club_goals', 'résultat']]

# Exemple d'utilisation
saison_specifique = '2023'  # Remplacez par la saison désirée
club_id_specifique = 1041  # OLYMPIQUE LYONNAIS
matchs_de_la_saison = games_of_the_season(data_dict['games'], club_id_specifique, saison_specifique, 'tous')
print(matchs_de_la_saison)


In [ ]:
def last_five_games(df_games, club_id, lieu='tous'):
    """
    Récupère les 5 derniers matchs d'un club spécifié en fonction du lieu du match.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.
    lieu (str) : 'domicile', 'exterieur', ou 'tous' pour filtrer les matchs.
    
    Retourne:
    Liste : Les identifiants (game_id) des 5 derniers matchs.
    """
    if lieu == 'domicile':
        matchs_du_club = df_games[df_games['home_club_id'] == club_id]
    elif lieu == 'exterieur':
        matchs_du_club = df_games[df_games['away_club_id'] == club_id]
    else: # 'tous'
        matchs_du_club = df_games[(df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)]
    
    matchs_du_club = matchs_du_club.sort_values(by='date', ascending=False).head(5)
    return matchs_du_club['game_id'].tolist()

# Exemple d'utilisation
club_id_specifique = 1041  # OLYMPIQUE LYONNAIS
derniers_game_ids = last_five_games(data_dict['games'], club_id_specifique, 'tous') # 'domicile', 'exterieur' ou 'tous'
print(derniers_game_ids)

In [ ]:
#last game id 
def last_game_id(df_games, club_id):
    """
    Récupère l'ID du dernier match joué par un club spécifié.

    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.

    Retourne:
    int : ID du dernier match joué par le club.
    """
    # Filtrer les matchs impliquant le club
    matchs_du_club = df_games[(df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)]
    
    # Trier par date en ordre décroissant et prendre le premier match
    dernier_match = matchs_du_club.sort_values(by='date', ascending=False).iloc[0]

    return dernier_match['game_id']

# Exemple d'utilisation
club_id_specifique = 1041  # Remplacez par l'ID du club désiré
dernier_match_id = last_game_id(data_dict['games'], club_id_specifique)
print("ID du dernier match de l'équipe", club_id_specifique, ":", dernier_match_id)


In [ ]:
def match_sheet(df_game_lineups, match_id, club_id):
    """
    Récupère tous les player_id qui ont participé à un match spécifique pour un club donné.
    
    Paramètres:
    df_game_lineups (DataFrame) : DataFrame des compositions d'équipe.
    match_id (int) : ID du match.
    club_id (int) : ID du club.
    
    Retourne:
    Liste : Les identifiants (player_id) des joueurs qui ont participé au match.
    """
    # Filtrer pour obtenir les lignes correspondant au match_id et club_id spécifiés
    lineup_du_match = df_game_lineups[(df_game_lineups['game_id'] == match_id) & (df_game_lineups['club_id'] == club_id)]
    
    # Récupérer les player_id
    joueurs = lineup_du_match['player_id'].tolist()

    return joueurs

# Exemple d'utilisation
match_id_specifique = 4094700
club_id_specifique = 1041  # OLYMPIQUE LYONNAIS
joueurs_participes = match_sheet(data_dict['game_lineups'], match_id_specifique, club_id_specifique)
print(joueurs_participes)

In [ ]:
def starting_lineup_match(match_id, club_id):
    """
    Récupère les player_id des joueurs qui étaient dans la composition de départ d'un match spécifique pour un club donné.
    
    Paramètres:
    df_game_lineups (DataFrame) : DataFrame des compositions d'équipe.
    match_id (int) : ID du match.
    club_id (int) : ID du club.
    
    Retourne:
    Liste : Les identifiants (player_id) des joueurs de la composition de départ.
    """
    df_game_lineups = data_dict['game_lineups']
    # Filtrer pour obtenir les lignes correspondant au match_id, club_id et type 'starting_lineup'
    lineup_du_match = df_game_lineups[(df_game_lineups['game_id'] == match_id) & 
                                      (df_game_lineups['club_id'] == club_id) & 
                                      (df_game_lineups['type'] == 'starting_lineup')]
    
    # Récupérer les player_id
    joueurs = lineup_du_match['player_id'].tolist()

    return joueurs

# Exemple d'utilisation
match_id_specifique = 4094700  # Remplacez par l'ID du match désiré
club_id_specifique = 1041  # OLYMPIQUE LYONNAIS
joueurs_titulaires = starting_lineup_match(match_id_specifique, club_id_specifique)
print(joueurs_titulaires)

In [ ]:
def market_value_player(df_player, player_id):
    """
    Récupère la valeur marchande d'un joueur spécifié.
    
    Paramètres:
    df_player_valuations (DataFrame) : DataFrame des évaluations des joueurs.
    player_id (int) : ID du joueur.
    
    Retourne:
    float : La valeur marchande du joueur en euros.
    """
    # Trouver l'entrée correspondant au player_id
    valeur_joueur = df_player[df_player['player_id'] == player_id]['market_value_in_eur']

    # Retourner la valeur marchande, ou None si le joueur n'est pas trouvé
    return valeur_joueur.iloc[0] if not valeur_joueur.empty else None

# Exemple d'utilisation
player_id_specifique = 955070  # Remplacez par l'ID du joueur désiré
valeur_marchande = market_value_player(data_dict['player_valuations'], player_id_specifique)
print(valeur_marchande)

In [ ]:
def mean_team_value(df_player, liste_player_id):
    """
    Calcule la valeur marchande moyenne d'une équipe.
    
    Paramètres:
    df_player_valuations (DataFrame) : DataFrame des évaluations des joueurs.
    liste_player_id (List[int]) : Liste des ID des joueurs.
    
    Retourne:
    float : Valeur marchande moyenne de l'équipe.
    """
    valeurs = [market_value_player(df_player, player_id) for player_id in liste_player_id]
    valeurs = [v for v in valeurs if v is not None]  # Exclure les valeurs None
    return sum(valeurs) / len(valeurs) if valeurs else 0

def sum_team_value(df_player, liste_player_id):
    """
    Calcule la somme des valeurs marchandes d'une équipe.
    
    Paramètres:
    df_player_valuations (DataFrame) : DataFrame des évaluations des joueurs.
    liste_player_id (List[int]) : Liste des ID des joueurs.
    
    Retourne:
    float : Somme des valeurs marchandes de l'équipe.
    """
    valeurs = [market_value_player(df_player, player_id) for player_id in liste_player_id]
    valeurs = [v for v in valeurs if v is not None]  # Exclure les valeurs None
    return sum(valeurs)

# Exemple d'utilisation
liste_player_id = [855134, 655006, 395237, 131225, 93720, 607223, 955070, 711544, 37838, 60561, 187718] 
moyenne_valeur_marchande = mean_team_value(data_dict['players'], liste_player_id)
somme_valeur_marchande = sum_team_value(data_dict['players'], liste_player_id)

print("Valeur marchande moyenne de l'équipe :", moyenne_valeur_marchande)
print("Somme des valeurs marchandes de l'équipe :", somme_valeur_marchande)

CREATION DU MODEL

In [ ]:
#Configurations et imports
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#pour cette première etape, on se base sur la forme de l'équipe et la valeur marchande des joueurs (indicatif de la qualité du joueur)
def calculer_forme(df_games, club_id, type):
    """Calcule la forme d'un club sur ses n derniers matchs avant la date du match."""
    # Filtrer les matchs précédents et trier par date
    matchs_precedents = games_of_the_season(df_games, club_id, 2022, type)
    
    # Calculer les points (exemple simplifié)
    points = 0
    for _, row in matchs_precedents.iterrows():
        if row['home_club_id'] == club_id:
            points += 3 if row['home_club_goals'] > row['away_club_goals'] else 1 if row['home_club_goals'] == row['away_club_goals'] else 0
        else:
            points += 3 if row['away_club_goals'] > row['home_club_goals'] else 1 if row['away_club_goals'] == row['home_club_goals'] else 0
    
    return points

In [ ]:
#on travaille seulement sur la saison 2022 pour eviter les traitements inutiles
#matchs_2023 = data_dict["games"][data_dict["games"]['season'] == 2023]
# Supposons que 'data_dict["games"]' est votre DataFrame de matchs
df_games = data_dict["games"]

# Filtrer pour obtenir les matchs de la saison 2022 et des compétitions spécifiées
competitions_ids = ['FR1', 'ES1', 'IT1', 'L1', 'GB1'] #5 grands championnats
df_matchs = df_games[(df_games['season'] == 2022)]# & (df_games['competition_id'].isin(competitions_ids))]

df_matchs

In [ ]:
# Création des caractéristiques pour chaque match
features = []
labels = []
df_games = data_dict["games"]
df_matchs = df_matchs
df_game_lineup = data_dict["game_lineups"]
df_player_valuations = data_dict["players"]

for _, row in df_matchs.iterrows():
    home_forme = calculer_forme(df_matchs, row['home_club_id'], "domicile")
    away_forme = calculer_forme(df_matchs, row['away_club_id'], "exterieur")
    home_valeur_marchande = mean_team_value(df_player_valuations, starting_lineup_match(row["game_id"], row['home_club_id']))
    away_valeur_marchande = mean_team_value(df_player_valuations, starting_lineup_match(row["game_id"], row['away_club_id'])) 
    
    features.append([home_forme, away_forme, home_valeur_marchande, away_valeur_marchande])
    
    # Créer le label
    if row['home_club_goals'] > row['away_club_goals']:
        labels.append(1)
    elif row['home_club_goals'] < row['away_club_goals']:
        labels.append(-1)
    else:
        labels.append(0)

# Convertir en DataFrame pour l'utilisation avec des modèles de machine learning
df_features = pd.DataFrame(features, columns=['home_forme', 'away_forme', 'home_valeur_marchande', 'away_valeur_marchande'])
df_labels = pd.Series(labels)

In [ ]:
print(df_labels)
print(df_features)

In [ ]:
# Supprimer les lignes contenant des valeurs NaN
df_features_clean = df_features.dropna()
df_labels_clean = df_labels[df_features_clean.index]
df_features_clean
df_labels_clean

EN UTILISANT UNE REGRESSION LOGISTIQUE

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df_features_clean, df_labels_clean, test_size=0.1, random_state=42)

# Création et entraînement du modèle
model = LogisticRegression()
model.fit(X_train, y_train)

# Évaluation du modèle
predictions = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))

In [ ]:
def calculer_probabilites_victoire(df_games, df_players, model, liste_matchs):
    """
    Calcule les probabilités de victoire pour une liste de matchs.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    df_players (DataFrame) : DataFrame des joueurs et de leurs valeurs marchandes.
    model (Model) : Modèle de machine learning entraîné.
    liste_matchs (List[Tuple[int, int]]) : Liste de tuples (home_club_id, away_club_id).
    
    Retourne:
    DataFrame : Probabilités de victoire pour chaque match.
    """
    # Liste pour stocker les caractéristiques de chaque match
    features_list = []
    
    # Préparer les caractéristiques pour chaque match
    for home_club_id, away_club_id in liste_matchs:
        home_forme = calculer_forme(df_games, home_club_id, "domicile")
        away_forme = calculer_forme(df_games, away_club_id, "exterieur")
        home_valeur_marchande = mean_team_value(df_players, starting_lineup_match(last_game_id(df_games, home_club_id), home_club_id))
        away_valeur_marchande = mean_team_value(df_players, starting_lineup_match(last_game_id(df_games, away_club_id), away_club_id))
        
        # Ajouter les caractéristiques pour le match courant à la liste
        features_list.append([home_club_id, away_club_id, home_forme, away_forme, home_valeur_marchande, away_valeur_marchande])
    
    # Créer un DataFrame avec toutes les caractéristiques
    df_features = pd.DataFrame(features_list, columns=['home_club_id', 'away_club_id', 'home_forme', 'away_forme', 'home_valeur_marchande', 'away_valeur_marchande'])
    
   # Préparer les caractéristiques pour la prédiction
    match_features = df_features[['home_forme', 'away_forme', 'home_valeur_marchande', 'away_valeur_marchande']]
    
    # Prédiction des probabilités avec le modèle pour chaque match
    probas = model.predict_proba(match_features)
    
    # Ajouter toutes les probabilités prédites au DataFrame
    for i, class_label in enumerate(model.classes_):
        df_features[f'prob_{class_label}'] = probas[:, i]
    
    # Sélectionner uniquement les colonnes 'home_club_id', 'away_club_id', et celles qui commencent par 'prob_'
    df_probabilites = df_features.filter(regex='^(home_club_id|away_club_id|prob_)')
    
    return df_probabilites

# Liste des matchs sous forme de tuples (home_club_id, away_club_id)
liste_matchs = [(1041, 583), (1041, 3524), (583, 3524), (1041, 162), (162, 1041), (418, 3302), (418, 12321), (31, 281), (281, 31), (418, 281), (281, 418)]  # Ajoutez d'autres matchs comme nécessaire

# Appel de la fonction et impression des résultats
df_probabilites = calculer_probabilites_victoire(df_games, df_players, model, liste_matchs)
print(df_probabilites)

In [ ]:
"""
1041 = OL
583 = PSG
3524 = Clermont
162 = Monaco
418 = Real Madrid
3302 = Almeria
12321 = Girona
31 = Liverpool
281 = ManchesterCity
"""

In [ ]:
import joblib
# Supposons que `model` est votre modèle entraîné
joblib.dump(model, 'model.pkl')
